In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tokenize/tokenizer.json
/kaggle/input/tokenize/tokenizer_config.json
/kaggle/input/tokenize/vocab.txt
/kaggle/input/kaggle-llm-science-exam/sample_submission.csv
/kaggle/input/kaggle-llm-science-exam/train.csv
/kaggle/input/kaggle-llm-science-exam/test.csv
/kaggle/input/berttrain/config.json
/kaggle/input/berttrain/pytorch_model.bin
/kaggle/input/bertmodel2/config.json
/kaggle/input/bertmodel2/pytorch_model.bin
/kaggle/input/tokenizetrain/tokenizer.json
/kaggle/input/tokenizetrain/tokenizer_config.json
/kaggle/input/tokenizetrain/special_tokens_map.json
/kaggle/input/tokenizetrain/vocab.txt


In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoModel, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForMultipleChoice

import pandas as pd
import torch
from transformers import BertForSequenceClassification
from transformers import BertForMultipleChoice, BertTokenizer
from transformers import BertForQuestionAnswering
import transformers
from transformers import AdamW
from torch.nn import CrossEntropyLoss

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
dft = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')

In [4]:
model_name = "allenai/scibert_scivocab_uncased"
model = AutoModelForMultipleChoice.from_pretrained('/kaggle/input/bertmodel2')
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/tokenize')

# オプティマイザーの設定
optimizer = AdamW(model.parameters(), lr=1e-5)

# 損失関数の設定
loss_fn = CrossEntropyLoss()

# 訓練データの長さ
n = len(dft['prompt'])

for epoch in range(10):  # エポック数は適宜調整してください
    total_loss = 0

    for a in range(n):
        prompt = dft.iloc[a]['prompt']
        choices = ["A", "B", "C", "D", "E"]
        choice_texts = dft.iloc[a][choices].values.tolist()

        # 入力データとラベルのエンコード
        encoded = tokenizer([prompt]*5, choice_texts, padding=True, return_tensors='pt', truncation=True, max_length=512)
        input_ids = encoded['input_ids'].view(1, 5, -1)
        attention_mask = encoded['attention_mask'].view(1, 5, -1)
        labels = torch.tensor([choices.index(dft.iloc[a]['answer'])])  # 'answer'カラムから正解ラベルを取得

        # 勾配をゼロに初期化
        optimizer.zero_grad()

        # フォワードパス
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # バックワードパスとパラメータ更新
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    #print(f"Epoch {epoch+1}, Loss: {total_loss/n}")

# モデルとトークナイザーの保存
tokenizer.save_pretrained('/kaggle/working/')
model.save_pretrained("/kaggle/working/")

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [5]:
df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv")
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
